File created by Jinghao Chen (jinghc2@uci.edu).

Import necessary packages.

In [1]:
import pickle
import torch
import mptools as mp
import numpy as np
np.random.seed(114514)

Load combined data from repository "/data".

In [2]:
pickle_in = open("data/data_comb.pkl","rb")
data_comb = pickle.load(pickle_in)

Calculate necessary values.

In [3]:
N_comb = data_comb.shape[1]
N_theta = 4+N_comb-1 # the number of unknown parameters

Shuffle and binary classified data.

In [4]:
X, Y = data_comb[:,:-1], data_comb[:,-1]   # get features & target
X, Y = mp.shuffleData(X,Y)       # reorder randomly rather than by class label

In [5]:
Xtr, Ytr = X[Y!=5,:], Y[Y!=5]
Ytr = np.heaviside(Ytr-5,-1)

Initialize $\theta$.

In [6]:
theta_ini = np.array([0.5,0.06,0.5,-3.26]+[0]*(N_comb-1)).reshape(N_theta,1) + 0.1*np.random.randn(N_theta,1)

Train the classifier to jointly learn $\theta$.

In [7]:
n_set = 500
multi_startup = 1
Theta = np.empty([n_set,N_theta])

In [8]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(n_set)):
    Xb, Yb = mp.bootstrapData(Xtr, Ytr)
    theta_ini = np.array([0.5,0.06,0.5,-3.26]+[0]*(N_comb-1)).reshape(N_theta,1) + multi_startup*0.1*np.random.randn(N_theta,1)
    learner = mp.jointClassifier()
    learner.classes = np.unique(Yb)
    learner.theta = torch.tensor(theta_ini,requires_grad = True,dtype=torch.float32)
    learner.train(Xb,Yb,initStep=1e-1,stopTol=1e-4,stopEpochs=100,alpha=10,plotOpt=False)
    Theta[i] = learner.theta.detach().numpy().reshape(1,N_theta)

  0%|                                                   | 0/500 [00:00<?, ?it/s]

First 4 values are reference and maximal transcription rates in ratio $\frac{\bar{\tau}}{\tau_{max}}$, concentration of CRP $C_c$, concentration of RNAP $C_r$ and CRP-RNAP interaction energy $\varepsilon_i$.

Save it if needed.

In [ ]:
# pickle_out = open("data/theta230731joint272param.pkl","wb")
# pickle.dump(theta_np, pickle_out)
# pickle_out.close()